In [1]:
import data.audio_dataset_v1 as data_utils
import models.inversion_v1 as model_utils
from abstract_model import AbstractModel

import torch
import torch.nn as nn
from torch import optim

from argparse import Namespace
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger

from ipywidgets import IntProgress

In [2]:
AudioDataset = data_utils.AudioDataset
InversionV1 = model_utils.InversionV1

In [3]:
audio_dataset = AudioDataset(root_dir='/scratch/prs392/incubator/data/LibriSpeech/train-clean-360')

for i in range(len(audio_dataset)):
    emb, spec, j = audio_dataset[i]
    print(emb.shape, spec.shape, j)
    m = InversionV1()
    pred = m(emb)
    print("Embeddings shape: " + str(emb.shape))
    print("Expected Spectrogram shape: " + str(spec.shape))
    print("Predicted Spectrogram shape: " + str(pred.shape))
    if i == 0:
        break

torch.Size([6144]) torch.Size([1, 128, 199]) tensor(122)
Embeddings shape: torch.Size([6144])
Expected Spectrogram shape: torch.Size([1, 128, 199])
Predicted Spectrogram shape: torch.Size([1, 1, 128, 199])


In [10]:
data_paths = {}
data_paths['train'] = '/scratch/prs392/incubator/data/LibriSpeech/train-clean-360'
data_paths['val'] = '/scratch/prs392/incubator/data/LibriSpeech/dev-clean'
data_paths['test'] = '/scratch/prs392/incubator/data/LibriSpeech/test-clean'

def optimizer(model_parameters, hparams):
    optimizer = optim.SGD(model_parameters, lr=hparams.lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, hparams.scheduler_epoch, hparams.scheduler_step_size)
    return [optimizer], [scheduler]

args = {
    'batch_size': 4, # Compulsory
    'lr': 0.02, # Compulsory
    'scheduler_epoch': 20, # Compulsory
    'scheduler_step_size': 0.1 # Compulsory
}

hparams = Namespace(**args)

In [11]:
hparams.lr

0.02

In [12]:
pl.seed_everything(123)

model = AbstractModel(
    hparams=hparams,
    data_paths = data_paths, 
    dataset_model = AudioDataset,
    model = InversionV1(), 
    criterion = nn.MSELoss(),
    optimizer = optimizer
)

checkpoint_path = f"/scratch/prs392/incubator/checkpoints/openl3_librispeech/inversion_v1/"

logger = TensorBoardLogger(checkpoint_path, name="overfitting")

trainer = pl.Trainer(logger = logger, 
                     default_root_dir=checkpoint_path,
                     max_epochs = 100,
                     gpus = -1,
                     distributed_backend = 'dp',
                     val_check_interval=0.25,
                     log_save_interval=1,
                     row_log_interval=1,
                     fast_dev_run=False,
                    num_sanity_val_steps=0,
                    overfit_batches = 0.001)     
trainer.fit(model)
trainer.test(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0,1,2]
Set SLURM handle signals.

  | Name      | Type        | Params
------------------------------------------
0 | model     | InversionV1 | 7 M   
1 | criterion | MSELoss     | 0     


Set SLURM handle signals.


--------------------------------------------------------------------------------
TEST RESULTS
{'test_loss': tensor(46.8243, device='cuda:0')}
--------------------------------------------------------------------------------



In [9]:
for batch in trainer.train_dataloader:
    x, y, i = batch
    print('----------')

----------
----------
----------
----------
----------
----------
----------
----------
----------
----------
----------
----------
----------
----------
----------
----------
----------
----------
----------
----------
----------
----------
----------
----------
----------
----------
----------
----------
----------
----------
----------
----------


KeyboardInterrupt: 